In [1]:
# Imports
import os
import logging

import pystac
import geopandas as gpd
from tempfile import TemporaryDirectory
from shapely.geometry import mapping

from eodag import setup_logging
from eodag.api.core import EODataAccessGateway

from eoreader.reader import Reader
from eoreader.stac import OPTICAL_STAC_EXTENSIONS

In [2]:
logger = logging.getLogger("eoreader")
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

In [3]:
prod_folder = os.path.join("/home", "data", "DATA", "PRODS")
paths = [
    # Landsat-8 OLI-TIRS collection 2
    os.path.join(prod_folder, "LANDSATS_COL2", "LC08_L1TP_200030_20201220_20210310_02_T1.tar"),
    # Landsat-5 TM collection 2    
    os.path.join(prod_folder, "LANDSATS_COL2", "LT05_L1TP_200030_20111110_20200820_02_T1.tar"),
    # Sentinel-2 L2A
    os.path.join(prod_folder, "S2", "PB 02.07+", "S2A_MSIL1C_20191215T110441_N0208_R094_T30TXP_20191215T114155.SAFE"),
]

In [4]:
# Create the reader
reader = Reader()

# Work in a temporary directory
tmp = TemporaryDirectory()


In [5]:
# Create STAC catalog
catalog_path = os.path.join(tmp.name, "catalog.json")
catalog = pystac.Catalog(
    id='SERTIT_101',
    description="SERTIT's Catalog",
    title='SERTIT Catalog',
    stac_extensions=OPTICAL_STAC_EXTENSIONS,
    href=catalog_path
)

In [6]:
# Add all the products into the STAC catalog
for path in paths:
    logger.info(f"*** {os.path.basename(path)} ***")

    # Open the product
    prod = reader.open(path, remove_tmp=True)

    # Get item
    item = prod.stac.create_item()

    # Add item to catalogue
    catalog.add_item(item)

*** LC08_L1TP_200030_20201220_20210310_02_T1.tar ***


No quicklook found in 20201220T104856_L8_200030_OLI_TIRS


*** LT05_L1TP_200030_20111110_20200820_02_T1.tar ***


*** S2A_MSIL1C_20191215T110441_N0208_R094_T30TXP_20191215T114155.SAFE ***


No quicklook found in 20191215T110441_S2_T30TXP_L1C_114155


In [7]:
# Save catalog
catalog.describe()
catalog.normalize_and_save(tmp.name, catalog_type=pystac.CatalogType.SELF_CONTAINED)

* <Catalog id=SERTIT_101>
  * <Item id=20201220T104856_L8_200030_OLI_TIRS>
  * <Item id=20111110T103612_L5_200030_TM>
  * <Item id=20191215T110441_S2_T30TXP_L1C_114155>


In [8]:
#With this notebook you are allowed to search into your STAC database and show image bbox on a map with leaflet.
#If you are working on Windows you will need to convert your href into linux path with "Transform_href_to_linux.py" script
dag = EODataAccessGateway()

# Decrease logging level
setup_logging(verbose=2)

# add the provider
dag.update_providers_config("""
stac_http_provider:
    search:
        type: StaticStacSearch
        api_endpoint: %s
    products:
        GENERIC_PRODUCT_TYPE:
            productType: '{productType}'
    download:
        type: HTTPDownload
        base_uri: %s
        flatten_top_dirs: True
        outputs_prefix: %s
""" % (catalog_path, tmp.name, tmp.name))

dag.set_preferred_provider("stac_http_provider")

2022-05-11 20:17:32,744-15s eodag.config                     [INFO    ] stac_http_provider: unknown provider found in user conf, trying to use provided configuration


In [9]:
#We can also look for datetime
all_products, _ = dag.search()

2022-05-11 20:17:32,839-15s eodag.core                       [INFO    ] No product type could be guessed with provided arguments


2022-05-11 20:17:33,458-15s eodag.core                       [INFO    ] Searching product type 'None' on provider: stac_http_provider


2022-05-11 20:17:33,567-15s eodag.plugins.crunch.filter_overlap [WARNING ] geometry not found in cruncher arguments, filtering disabled.


2022-05-11 20:17:33,571-15s eodag.core                       [INFO    ] Found 3 result(s) on provider 'stac_http_provider'


In [10]:
#Here we are able to draw bbox in order to find every image that are going to cross this bbox
aoi_path = os.path.join("/home", "data", "DATA", "AOIs", "DAX.geojson")
aoi = gpd.read_file(aoi_path)
aoi_geojson = mapping(aoi.geometry.values[0])

query_args = {"start": "2020-05-01", "end": "2022-05-06", "geom": aoi.geometry.values[0]}
query_products, _ = dag.search(**query_args)

2022-05-11 20:17:33,590-15s eodag.core                       [INFO    ] No product type could be guessed with provided arguments


2022-05-11 20:17:33,591-15s eodag.core                       [INFO    ] Searching product type 'None' on provider: stac_http_provider


2022-05-11 20:17:33,698-15s eodag.plugins.crunch.filter_date [INFO    ] Finished filtering products. 1 resulting products


2022-05-11 20:17:33,699-15s eodag.plugins.crunch.filter_overlap [INFO    ] Finished filtering products. 1 resulting products


2022-05-11 20:17:33,701-15s eodag.core                       [INFO    ] Found 1 result(s) on provider 'stac_http_provider'


In [11]:
import folium

# Create a map zoomed over the search area
fmap = folium.Map((43.2, -1.05), zoom_start=7)


# Add a layer green layer for the query over the AOI
folium.GeoJson(
    data=all_products.as_geojson_object(),
    tooltip = "All products stored in the catalog",
    style_function=lambda x: {'color': 'green'}
).add_to(fmap)


# Add a layer green layer for the query over the AOI
folium.GeoJson(
    data=query_products.as_geojson_object(),
    tooltip = "Retrieved products with the query",
    style_function=lambda x: {'color': 'red'}
).add_to(fmap)

# Add a layer blue layer for the AOI
folium.GeoJson(
    data=aoi_geojson,
    tooltip = "DAX AOI",
    style_function=lambda x: {'color': 'blue'}
).add_to(fmap)

fmap

In [12]:
# Clean the tmp directory
tmp.cleanup()